In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchtext
from torchtext.legacy import data, datasets

import random
import numpy as np
import time
import spacy


torch.__version__, torchtext.__version__, spacy.__version__, np.__version__

('1.8.0+cu101', '0.9.0', '2.2.4', '1.19.5')

## Preparing Data

In [2]:
%%time

# set random seed for reproducibility
SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# download and split dataset (train, valid, test)
TEXT = data.Field(
    tokenize='spacy', tokenizer_language='en_core_web_sm', batch_first=True
)
LABEL = data.LabelField(dtype=torch.float)

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)
train_data, valid_data = train_data.split(random_state=random.seed(SEED))

aclImdb_v1.tar.gz:   0%|          | 0.00/84.1M [00:00<?, ?B/s]

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:05<00:00, 14.4MB/s]


CPU times: user 2min 2s, sys: 16.5 s, total: 2min 18s
Wall time: 2min 27s


In [3]:
# check the type and size of dataset
print(f'>>> type : {type(train_data)}')
print(f'>>> Number of training examples: {len(train_data)}')   # 17500 (35%)
print(f'>>> Number of validation examples: {len(valid_data)}') # 7500  (15%)
print(f'>>> Number of testing examples: {len(test_data)}')     # 25000 (50%)
print()

# check one sample data
tmp_ex = train_data.examples[0]
tmp_dict = vars(tmp_ex)
print('< example data >')
print('>>> type :', type(tmp_ex))
for key in tmp_dict:
  print(f'>>> {key} : {tmp_dict[key]}')

>>> type : <class 'torchtext.legacy.data.dataset.Dataset'>
>>> Number of training examples: 17500
>>> Number of validation examples: 7500
>>> Number of testing examples: 25000

< example data >
>>> type : <class 'torchtext.legacy.data.example.Example'>
>>> text : ['JUST', 'CAUSE', 'showcases', 'Sean', 'Connery', 'as', 'a', 'Harvard', 'law', 'prof', ',', 'Kate', 'Capshaw', '(', 'does', 'she', 'still', 'get', 'work', '?', ')', 'as', 'his', 'wife', '(', 'slight', 'age', 'difference', ')', 'and', 'Lawrence', 'Fishburne', 'as', 'a', 'racist', 'southern', 'cop', '(', '!', ')', 'and', 'Ed', 'Harris', 'in', 'a', 'totally', 'over', 'the', 'top', 'rendition', 'of', 'a', 'fundamentalist', 'southern', 'serial', 'killer.<br', '/><br', '/>Weird', 'casting', ',', 'but', 'the', 'movie', 'plays', 'serious', 'mindf', '*', '*', 'with', 'the', 'audience', '.', '(', 'do', "n't", 'read', 'if', 'you', 'ever', 'intend', 'to', 'seriously', 'watch', 'this', 'film', 'or', 'to', 'ever', 'watch', 'this', 'film', '

In [4]:
%%time

# build vocabulary
MAX_VOCAB_SIZE = 25_000
TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)
LABEL.build_vocab(train_data)

print('\n')
print(f">>> Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f">>> Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")
print(f">>> Top 20 common tokens :{TEXT.vocab.freqs.most_common(20)}")
print()
print('<itos and stoi>')
print('>>> itos :', TEXT.vocab.itos[:10])
print('>>> stoi :', LABEL.vocab.stoi)

.vector_cache/glove.6B.zip: 862MB [02:41, 5.35MB/s]                           
100%|█████████▉| 399215/400000 [00:17<00:00, 23904.73it/s]



>>> Unique tokens in TEXT vocabulary: 25002
>>> Unique tokens in LABEL vocabulary: 2
>>> Top 20 common tokens :[('the', 202538), (',', 192715), ('.', 165259), ('and', 109176), ('a', 108967), ('of', 101099), ('to', 93437), ('is', 76014), ('in', 61519), ('I', 54176), ('it', 53451), ('that', 49386), ('"', 44430), ("'s", 43117), ('this', 42352), ('-', 37369), ('/><br', 35648), ('was', 35029), ('as', 30258), ('with', 29733)]

<itos and stoi>
>>> itos : ['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is']
>>> stoi : defaultdict(None, {'neg': 0, 'pos': 1})
CPU times: user 41.9 s, sys: 8.43 s, total: 50.3 s
Wall time: 3min 25s


In [5]:
# create the iterators
BATCH_SIZE = 64
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), batch_size=BATCH_SIZE, device=device
)

display(device, type(train_iterator), len(train_iterator), len(train_data)/BATCH_SIZE)

device(type='cpu')

torchtext.legacy.data.iterator.BucketIterator

274

273.4375

## Build the Model

In [8]:
class CNN(nn.Module):
  def __init__(self, vocab_size, embedding_dim, pad_idx, n_filters, filter_sizes, output_dim, dropout):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim, pad_idx)
    self.conv_0 = nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(filter_sizes[0], embedding_dim))
    self.conv_1 = nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(filter_sizes[1], embedding_dim))
    self.conv_2 = nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(filter_sizes[2], embedding_dim))
    self.dropout = nn.Dropout(dropout)
    self.fc = nn.Linear(len(filter_sizes)*n_filters, output_dim)
  
  def forward(self, text):
    # text = [batch size, sent len]
    print(text.shape)

    embedded = self.embedding(text)
    embedded = embedded.unsqueeze(1)
    # embedded = [batch size, 1, sent len, emb dim]
    print(embedded.shape)

    conv0 = F.relu(self.conv_0(embedded).squeeze(3))
    conv1 = F.relu(self.conv_1(embedded).squeeze(3))
    conv2 = F.relu(self.conv_2(embedded).squeeze(3))
    # conv = [batch size, n_filters, sent len-filter_sizes[n]+1]
    print(conv0.shape, conv1.shape, conv2.shape)

    pool0 = F.max_pool1d(conv0, conv0.shape[2]).squeeze(2)
    pool1 = F.max_pool1d(conv1, conv1.shape[2]).squeeze(2)
    pool2 = F.max_pool1d(conv2, conv2.shape[2]).squeeze(2)
    # pool = [batch size, n_filters]
    print(pool0.shape, pool1.shape, pool2.shape)

    cat = self.dropout(torch.cat((pool0, pool1, pool2), dim=1))
    # cat = [batch size, n_filters*len(filter_sizes)]
    print(cat.shape)

    out = self.fc(cat)
    # out = [batch size, output dim]
    print(out.shape)
    
    return out

In [9]:
# shape test code
model = CNN(25000, 100, 0, 64, (3, 4, 5), 1, 0.5)

arr = torch.randint(low=0, high=25000, size=(1, 10), dtype=torch.long)
if 0 in arr: print(arr)
model(arr).sigmoid()

torch.Size([1, 10])
torch.Size([1, 1, 10, 100])
torch.Size([1, 64, 8]) torch.Size([1, 64, 7]) torch.Size([1, 64, 6])
torch.Size([1, 64]) torch.Size([1, 64]) torch.Size([1, 64])
torch.Size([1, 192])
torch.Size([1, 1])


tensor([[0.6591]], grad_fn=<SigmoidBackward>)